similarity_to(spec2, round_precision=0)[source]

    Compares two spectra and returns cosine

Parameters

    spec2 (Spectrum) – another pymzml spectrum that is compared to the current spectrum.

Keyword Arguments
    
    round_precision (int) – precision mzs are rounded to, i.e. round( mz, round_precision )

Returns
    
    value between 0 and 1, i.e. the cosine between the two spectra.

Return type
    
    cosine (float)

highest_peaks(n)[source]

Function to retrieve the n-highest centroided peaks of the spectrum.

In [106]:
import pymzml
import matplotlib.pyplot as plt
import numpy as np

import pyteomics
from pyteomics import mzml, auxiliary

import plotly.graph_objects as go

import tqdm

In [85]:
path = 'D:/UW/massmotif/mzml/DRO_DIE_1ppm_29h_exp1_method_1.mzML'

In [3]:
f=mzml.MzML(path)

In [32]:
def get_scans(path, ms_all = False, ms_lv = 1):
    run = pymzml.run.Reader(path)
    scans = []
    if ms_all == False:
        for scan in run:
            if scan.ms_level == ms_lv:
                scans.append(scan)
    elif ms_all == True:
        for scan in run:
            scans.append(scan)
            
    return scans

In [86]:
ms2s = get_scans(path, False, 2)

In [37]:
def motif_seek(ms2_scans, motifs, error = 0.002, noise_level = 50):
    motif_result = []
    
    motif_range = []
    for motif in motifs:
        motif_range.append([motif - error, motif + error])
    
    for scan in ms2_scans:
        precursor = scan.selected_precursors[0]['mz']
        drop_index = np.argwhere(scan.i <= noise_level)
        scan.i = np.delete(scan.i, drop_index)
        scan.mz = np.delete(scan.mz, drop_index)
        
        frag = scan.mz[scan.mz < precursor]
        #frag_i = scan.i[: len(frag)] # In case need it
        neutral_loss = precursor - frag
        
        mtf_count = 0
        for mtf in motif_range:
            mtf_hit = neutral_loss[(mtf[0] < neutral_loss) & (neutral_loss < mtf[1])]
            if len(mtf_hit) > 0:
                mtf_count += 1
        
        if mtf_count == len(motifs):
            motif_result.append([scan.selected_precursors[0]['mz'], round(scan.scan_time[0],2), scan.ID])
        
    #motif_seek = list(set(motif_result)) # if merge is needed
        
    return motif_result

In [87]:
motif = [41.0265, 18.0105]
motif_seek(ms2s, motif, error = 0.002)

[[308.162155151367, 7.08, 424922], [308.156855265299, 8.23, 493790]]

In [92]:
def find_scan(ms2s, scanid, interactive = True):
    
    for scan in ms2s:
        if scan.ID == scanid:
            break
    
    print('Precursor m/z: {:0.2f}, Scan time: {:0.1f} minute'.format(scan.selected_precursors[0]['mz'], scan.scan_time[0]))
    
    mz = scan.mz
    ints = scan.i
    
    if interactive == True:
        plt.clf()
        fig = go.Figure([go.Bar(x=mz, y=ints, marker_color = 'red', width = 0.5,
                        hovertemplate =
                        'Int: %{y}'+
                        '<br>m/z: %{x}<br>')])
        fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(0,0,0)',
                  marker_line_width=0.5, opacity=1)
        fig.update_layout(
                template = 'simple_white',
                width = 1000,
                height = 600,
                xaxis = dict(title = 'm/z ratio',
                        rangeslider=dict(
            visible = True
        )),
                yaxis = dict(
                    title = 'Intensity'))
        fig.show()
    
    elif interactive == False:
        plt.figure(figsize=(10,5))
        plt.bar(mz, ints, width = 1.0)
        plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
        plt.xlabel('m/z')
        plt.ylabel('Intensity')
        plt.title('MS1 spectrum')

    return 

In [93]:
test1 = find_scan(ms2s, 424922)

Precursor m/z: 308.16, Scan time: 7.1 minute


<Figure size 432x288 with 0 Axes>

In [124]:
def cos_one(input_scan, ms2_scans, score_thres = 0.9):
    count = 0
    for scan in ms2_scans:
        score = input_scan.similarity_to(scan, round_precision=0)
        if score >= score_thres:
            count +=1
            print(count)
    print('one scan finished')
                        
    return count

In [70]:
f[55]

{'index': 55,
 'id': 'scanId=13414',
 'defaultArrayLength': 5288,
 'scanList': {'count': 1,
  'scan': [{'scanWindowList': {'count': 1,
     'scanWindow': [{'scan window lower limit': 50.02774685283 m/z,
       'scan window upper limit': 699.154243725482 m/z}]},
    'scan start time': 0.223433333333 minute}],
  'no combination': ''},
 'precursorList': {'count': 1,
  'precursor': [{'spectrumRef': 'scanId=12593',
    'isolationWindow': {'isolation window target m/z': 196.992283472522 m/z},
    'selectedIonList': {'count': 1,
     'selectedIon': [{'selected ion m/z': 196.992283472522 m/z,
       'peak intensity': 50338.708622932434 number of detector counts}]},
    'activation': {'beam-type collision-induced dissociation': '',
     'collision energy': 20.0 electronvolt}}]},
 'negative scan': '',
 'base peak m/z': 87.001612633378 m/z,
 'base peak intensity': 39120.7891 number of detector counts,
 'total ion current': 381039.188 number of detector counts,
 'ms level': 2,
 'MSn spectrum': '',